In [3]:
!pip install safetensors

In [5]:
!pip install git+https://github.com/huggingface/diffusers.git --force-reinstall

  Cloning https://github.com/huggingface/diffusers.git to /private/var/folders/0h/b_lc_w711c15h0gfcm2r5nm00000gr/T/pip-req-build-sbablcbv
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers.git /private/var/folders/0h/b_lc_w711c15h0gfcm2r5nm00000gr/T/pip-req-build-sbablcbv
  Resolved https://github.com/huggingface/diffusers.git to commit 0de35e4a5204d2a94edc5ab108b5061a7a7013e8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached requests-2.31.0-py3-none-any.whl (62 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 56.5 MB/s eta 0:00:0000:0100:01
  Using cached Pillow-10.0.0-cp310-cp310-macosx_11_0_arm64.whl (3.1 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.7 MB/s eta 0:00:00
  Using cached filelock-3.12.2-py3-none-any.whl (10 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.3/289.3 kB 31.9 MB/s eta 0:00:00
   

In [1]:
!pip install runpod

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 1.1 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 2.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 48.9 MB/s eta 0:00:0000:010:01
  Using cached httpx-0.24.1-py3-none-any.whl (75 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.5/374.5 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 57.4 MB/s eta 0:00:00


  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Attempting uninstall: h11
    Found existing installation: h11 0.12.0
    Uninstalling h11-0.12.0:
      Successfully uninstalled h11-0.12.0
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.7
    Uninstalling pydantic-1.10.7:
      Successfully uninstalled pydantic-1.10.7
  Attempting uninstall: httpcore
    Found existing installation: httpcore 0.13.7
    Uninstalling httpcore-0.13.7:
      Successfully uninstalled httpcore-0.13.7
  Attempting uninstall: botocore
    Found existing installation: botocore 1.29.127
    Uninstalling botocore-1.29.127:
      Successfully uninstalled botocore-1.29.127
  Attempting uninstall: httpx
    Found existing installation: httpx 0.18.2
    Uninstalling httpx-0.18.2:
      Successfully uninstalled httpx-0.18.2
  Attempting uninstall: fastapi
    Found existing installation: fastapi 0.95.2
    Uninstalling fastapi-0.95.2:
      Successfully uninstalled fastapi-0.95.2
 

In [3]:
!pip install --upgrade fastapi

In [4]:
import torch
from diffusers import StableDiffusionPipeline
from diffusers import StableDiffusionXLPipeline
from diffusers import DDIMScheduler, EulerAncestralDiscreteScheduler, DPMSolverMultistepScheduler
import runpod
import subprocess
import nest_asyncio
import base64
from PIL import Image
from io import BytesIO
#nest_asyncio.apply()

def gen_image(jobs):
    job_input = jobs["input"]
    
    prompt = job_input["prompt"]
    negative_prompt = ""

    #nprompt = """UnrealisticDream {neg}"""
    nprompt = "ugly, deformed, noisy, blurry, distorted, grainy"
    pprompt = """Portrait photo of (Breathtaking) wet and (curvy) Kerala woman with (plump big belly:1.2) in rainy beach, 
    wearing (wet) saree, 
    {pos}, with cleavage and (curvy fat hips) exposed"""
    negative_prompt = nprompt.format(neg = negative_prompt)
    prompt = pprompt.format(pos = prompt)
    pipe = StableDiffusionXLPipeline.from_single_file("/Users/azsheikh/stable-diffusion-webui/models/Stable-diffusion/nightvisionXLPhotorealisticPortrait_beta0681Bakedvae.safetensors", 
                                                   safety_checker = None, requires_safety_checker = False, 
                                                   use_safetensors=True, custom_pipeline="lpw_stable_diffusion_xl")
    #pipe.unet.load_attn_procs(lora_path)
    pipe = pipe.to("mps")
    pipe.enable_attention_slicing()

    #pipe.load_textual_inversion("embeddings/UnrealisticDream.pt")
    #pipe.scheduler.config["use_karras_sigmas"] = True
    pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)
    pipe.safety_checker = None
    pipe.requires_safety_checker = False

        #protovision - 896x1152 and nightvision 832x1216
    image = pipe(prompt= prompt, negative_prompt=negative_prompt, num_images_per_prompt = 1, 
                 guidance_scale = 3, height = 1216, width = 832, num_inference_steps=35).images
    
    image[0].save(str(job_input["id"])+".png")
    img_name = str(job_input["id"])+".png"

    cmd = ['python', 'GFPGAN/inference_gfpgan.py', '-i', img_name, '-o', 'results', '-v', '1.3', '-s', '2']

    try:
        subprocess.run(cmd, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Oops! Something went wrong: {e}")

    # Open image file
    with open("results/restored_imgs"+img_name, "rb") as image_file:
        # Read the file
        image_data = image_file.read()

        # Encode as base64
        base64_image = base64.b64encode(image_data).decode("utf-8")

    # Now, `base64_image` contains the base64 string representation of your image


    return base64_image

runpod.serverless.start({"handler": gen_image})

--- Starting Serverless Worker |  Version 1.1.3 ---
INFO   | Using test_input.json as job input.
DEBUG  | Retrieved local job: {'input': {'prompt': 'Chubby Kerala Wonder Woman'}, 'id': 'local_test'}
INFO   | local_test | Started


  0%|          | 0/35 [00:00<?, ?it/s]

/Users/azsheikh/anaconda3/lib/python3.10/site-packages/diffusers/schedulers/scheduling_euler_ancestral_discrete.py:259: UserWarning: MPS: nonzero op is supported natively starting from macOS 13.0. Falling back on CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/Indexing.mm:218.)
  index_candidates = (self.timesteps == timestep).nonzero()
/Users/azsheikh/anaconda3/lib/python3.10/site-packages/torch/nn/functional.py:3931: UserWarning: MPS: passing scale factor to upsample ops is supported natively starting from macOS 13.0. Falling back on CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/UpSample.mm:233.)
  return torch._C._nn.upsample_nearest2d(input, output_size, scale_factors)


DEBUG  | local_test | Handler output: [<PIL.Image.Image image mode=RGB size=832x1216 at 0x29D3CF3D0>]
DEBUG  | local_test | run_job return: {'output': [<PIL.Image.Image image mode=RGB size=832x1216 at 0x29D3CF3D0>]}
INFO   | Job local_test completed successfully.
INFO   | Job result: {'output': [<PIL.Image.Image image mode=RGB size=832x1216 at 0x29D3CF3D0>]}
INFO   | Local testing complete, exiting.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /Users/azsheikh/anaconda3/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3460 in  │
│ run_code                                                                                         │
│                                                                                                  │
│   3457 │   │   │   │   if async_:                                                                │
│   3458 │   │   │   │   │   await eval(code_obj, self.user_global_ns, self.user_ns)               │
│   3459 │   │   │   │   else:                                                                     │
│ ❱ 3460 │   │   │   │   │   exec(code_obj, self.user_global_ns, self.user_ns)                     │
│   3461 │   │   │   finally:                                                                      │
│   3462 │   │   │   │   # Reset our crash handler in place                                        │
│   3463 │   │   │   │   sys.excepthook = old_excepthook                                           │
│                                                                                                  │
│ in <module>:40                                                                                   │
│                                                                                                  │
│   37 │   │   │   │    guidance_scale = 3, height = 1216, width = 832, num_inference_steps=35)    │
│   38 │   return image                                                                            │
│   39                                                                                             │
│ ❱ 40 runpod.serverless.start({"handler": gen_image})                                             │
│   41                                                                                             │
│                                                                                                  │
│ /Users/azsheikh/anaconda3/lib/python3.10/site-packages/runpod/serverless/__init__.py:130 in      │
│ start                                                                                            │
│                                                                                                  │
│   127 │   │   )                                                                                  │
│   128 │                                                                                          │
│   129 │   else:                                                                                  │
│ ❱ 130 │   │   worker.main(config)                                                                │
│   131                                                                                            │
│                                                                                                  │
│ /Users/azsheikh/anaconda3/lib/python3.10/site-packages/runpod/serverless/worker.py:130 in main   │
│                                                                                                  │
│   127 │   If running on RunPod, the worker loop is created.                                      │
│   128 │   """                                                                                    │
│   129 │   if _is_local(config):                                                                  │
│ ❱ 130 │   │   asyncio.run(rp_local.run_local(config))                                            │
│   131 │                                                                                          │
│   132 │   else:                                                                                  │
│   133 │   │   try:                                                                               │
│                                                                                                  │
│ /Users/azsheikh/anaconda3/lib/python3.10/site-packages/nest

In [12]:
image[0].save("photo_of_astronaut.png")


In [13]:
!python GFPGAN/inference_gfpgan.py -i photo_of_astronaut.png -o results -v 1.3 -s 2

/Users/azsheikh/anaconda3/lib/python3.10/site-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(
/Users/azsheikh/My_img_App/GFPGAN/inference_gfpgan.py:63: UserWarning: The unoptimized RealESRGAN is slow on CPU. We do not use it. If you really want to use it, please modify the corresponding codes.
  warnings.warn('The unoptimized RealESRGAN is slow on CPU. We do not use it. '
/Users/azsheikh/anaconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/azsheikh/anaconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Ar

In [18]:
!pip show diffusers


Name: diffusers
Version: 0.21.0.dev0
Summary: State-of-the-art diffusion in PyTorch and JAX.
Home-page: https://github.com/huggingface/diffusers
Author: The HuggingFace team
Author-email: patrick@huggingface.co
License: Apache
Location: /Users/azsheikh/anaconda3/lib/python3.10/site-packages
Requires: filelock, huggingface-hub, importlib-metadata, numpy, Pillow, regex, requests, safetensors
Required-by: lycoris-lora


In [1]:
import subprocess

# Run pip freeze command and capture the output
result = subprocess.run(["pip", "freeze"], stdout=subprocess.PIPE, text=True)

# Write the output to requirements.txt
with open("requirements.txt", "w") as f:
    f.write(result.stdout)

print("requirements.txt has been generated!")


requirements.txt has been generated!
